In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [1]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines_201901_v2.0_codesofconduct_en.pdf'
paper_name = "Guidelines 1/2019 on Codes of codes and Monitoring Bodies under Regulation 2016/679"
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/codes.md"

In [3]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 4, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/codes.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [3]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'^(\d+) (?!\.)(.+)$')
subsection_pattern = re.compile(r'^(\d+\.\d+)\s*(.*)') 
# subpoint_pattern = re.compile(r'^(\d+)\.\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^APPENDIX (\d+) - (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, section_reference])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif annex_match:
            match = annex_match
            annex_started = True
            section = "APPENDIX " + match.group(1)
            subsection = ""
            point = ""
            heading = True
            text = match.group(2)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, section_reference])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)

#df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
# Remove my note about the table
df = df[df["section_reference"] != ''] 

In [4]:
df[200:220]
#df.iloc[192]["text"]
#df[df["text"] == " "]

,section,subsection,point,heading,text,section_reference
205,,,,False,(Paragraph 20),APPENDIX 3
206,,,,False,2.Are you an association or other body represe...,APPENDIX 3
207,,,,False,(Paragraph 21),APPENDIX 3
208,,,,False,3.Have you provided details in your submission...,APPENDIX 3
209,,,,False,representative body that is capable of underst...,APPENDIX 3
210,,,,False,22),APPENDIX 3
211,,,,False,4.Have you clearly defined the processing acti...,APPENDIX 3
212,,,,False,which the code is intended to address? (Paragr...,APPENDIX 3
213,,,,False,5.Have you identified the territorial scope of...,APPENDIX 3
214,,,,False,(where relevant)? (Paragraph 24),APPENDIX 3


In [5]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 6 augmented with footnotes
Row 7 augmented with footnotes
Row 9 augmented with footnotes
Row 12 augmented with footnotes
Row 14 augmented with footnotes
Row 15 augmented with footnotes
Row 17 augmented with footnotes
Row 33 augmented with footnotes
Row 35 augmented with footnotes
Row 41 augmented with footnotes
Row 57 augmented with footnotes
Row 58 augmented with footnotes
Row 59 augmented with footnotes
Row 62 augmented with footnotes
Row 73 augmented with footnotes
Row 75 augmented with footnotes
Row 77 augmented with footnotes
Row 79 augmented with footnotes
Row 83 augmented with footnotes
Row 91 augmented with footnotes
Row 98 augmented with footnotes
Row 101 augmented with footnotes
Row 102 augmented with footnotes
Row 103 augmented with footnotes
Row 105 augmented with footnotes
Row 109 augmented with footnotes
Row 110 augmented with footnotes
Row 112 augmented with footnotes
Row 114 augmented with footnotes
Row 117 augmented with footnotes
Row 119 augmented with footnotes
R

In [6]:
file = "../../inputs/documents/codes.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [11]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.codes
importlib.reload(gdpr_rag.documents.codes)
from gdpr_rag.documents.codes import Codes

path_to_manual_as_csv_file = "../../inputs/documents/codes.parquet"

doc = Codes(path_to_manual_as_csv_file)


In [14]:
from IPython.display import Markdown, display

section = "1"
#section = "12.8"

#section = "APPENDIX 4"



print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))



1 INTRODUCTION


# 1 INTRODUCTION

1. Regulation 2016/679[^1] ("the GDPR") came into effect on 25 May 2018. One of the main objective of the GDPR is to provide a consistent level of data protection throughout the European Union and to prevent divergences hampering the free movement of personal data within the internal market.[^2] The GDPR also introduces the principle of accountability, which places the onus on data controllers to be responsible for, and be able to demonstrate compliance with the Regulation.[^3] The provisions under Articles 40 and 41 of the GDPR in respect of codes of conduct ("codes")represent a practical, potentially cost effective and meaningful method to achieve greater levels of consistency of protection for data protection rights. Codes can act as a mechanism to demonstrate compliance with the GDPR.[^4] Notably, they can help to bridge the harmonisation gaps that may exist between Member States in their application of data protection law.[^5] They also provide an opportunity for particular sectors to reflect upon common data processing activities and to agree to bespoke and practical data protection rules, which will meet the needs of the sector as well as the requirements of the GDPR.[^6]

2. Member States, Supervisory Authorities, the European Data Protection Board ("the Board") and the European Commission ("the Commission") are obliged to encourage the drawing up of codes to contribute to the proper application of the Regulation.[^7] These guidelines will support and facilitate "code owners" in drafting, amending or extending codes.

  
[^1]: The General Data Protection Regulation (EU) 2016/679 of the European Parliament and of the Council of 27April 2016 on the protection of natural persons with regard to the processing of personal data and on the free movement of such data, and repealing Directive 95/46/EC.  
[^2]: See Recital 13 of the GDPR.  
[^3]: See Article 5(2) of the GDPR.  
[^4]: See for example Articles 24(3) and 28(5) and 32(3). A code of conduct may also be used by data processors to demonstrate sufficient guarantees that their processing is compliant with the GDPR (See Article 28(5)).  
[^5]: See Recitals 77, 81, 98, 99, 148, 168 and Articles 24, 28, 35, 40, 41, 46, 57, 64, 70 of the GDPR. This is particularly the case where a code relates to processing activities in several Member States.  
[^6]: Codes do not necessarily need to be confined or limited to a specific sector. For example, a code could apply to separate sectors who have a common processing activity which share the same processing characteristics and needs. Where a code is cross-sectoral in its application, more than one monitoring body may be appointed under that code. However, where this is the case the code should make it absolutely clear as to the scope of that monitoring body's functions, in other words by specifying the sectors in respect of which each monitoring body will perform its functions under Article 41 and the oversight mechanisms available to each monitoring body. In This regard, the relevant sections of these guidelines which set out the responsibilities, obligations and accreditation requirements in relation to monitoring bodies apply individually to each such monitoring body appointed under the code.  
[^7]: Article 40(1) of the GDPR.